In [ ]:
%matplotlib inline

import pandas as pd
import random
import numpy as np
from scipy import stats

import statsmodels.api as sm
import matplotlib.pyplot as plt

## Does observing race cause police to pull over more minority drivers?

### Potential Outcomes

* The variable $Y_i$ denotes whether driver $i$ was pulled over, 
* The variable $M_i$ specifies whether driver $i$  was a minority driver (1).
* The variable $Y_{1i}$ specifies what the outcome would have been had the officer observed that driver $i$ was a minority driver.
* The variable $Y_{0i}$ specifies what the outcome would have been had the officer observed that driver $i$ was not a minority driver.

The notation below summarizes the potential outcomes for driver $i$:

$$\begin{equation}
Y_i=\begin{cases}
Y_{1i} \quad if\quad  M_i=1\\
Y_{0i} \quad if\quad M_i=0
\end{cases}
\end{equation}
$$

### Observed Difference in Stop Rate

The observed difference in stop rate is the difference between the proportion of minority drivers pulled over and the proportion of non-minority drivers pulled over (left-hand side). Using the identities of conditional probability, this quantity decomposes into two terms:

$$ \begin{eqnarray} E[Y_i|M_i=1]-E[Y_i|M_i=0]=&E[Y_{1i}|M_i=1]-E[Y_{0i}|M_i=1] \\ &+E[Y_{0i}|M_i=1]-E[Y_{0i}|M_i=0] \end{eqnarray} $$

The two terms have the following interpretations:

$$ E[Y_{0i}|M_i=1]-E[Y_{0i}|M_i=0] \quad \mbox{(selection bias)}$$ 

$$ E[Y_{1i}|M_i=1]-E[Y_{0i}|M_i=1] \quad \mbox{(effect of race on likelihood of stop)}$$ 

### Probability of Stop

We would like to estimate the effect of observed race on the likelihood of a traffic stop. 

The Null Hypothesis states that observed race has no effect on the likelihood of a stop:

$$H_0: E[Y_{1i}|M_i=1]-E[Y_{0i}|M_i=1]=0$$

While the Alternative Hypothesis states otherwise:

$$H_1: E[Y_{1i}|M_i=1]-E[Y_{0i}|M_i=1]\neq 0$$

## Simulations

We are going to test this hypothesis when we know the data generating process exactly (i.e. by generating the data).

### Simulation 1: Assign probability of stop from same distribution

$$  Y_i=f(M_i,\varepsilon_i) $$
$$ \mathbf{\varepsilon} \sim N(\mathbf{0},\mathbf{1}) $$


In [ ]:
NUM_DRIVERS = 100000
PCT_MINORITY = 0.1
STOP_THRESH =  -2

n_minority = int(NUM_DRIVERS * PCT_MINORITY)
n_notminor = int(NUM_DRIVERS * (1 - PCT_MINORITY))

# Generate data
M = np.array([0] * n_notminor + [1] * n_minority) # generate minority variable
N = np.random.normal(0, 1, size=NUM_DRIVERS) # generate error terms
df = pd.DataFrame({'Minority': M, 'StopLikelihood': N})

df['Stop'] = (df['StopLikelihood'] < STOP_THRESH).astype(int) # a stop occurs...
df.head()

Stop rates by minority vs not minority:

In [ ]:
df.pivot_table(values='Stop', index='Minority', aggfunc='mean')

Testing the average difference in stop rates between the two groups:

In [ ]:
minority = df.loc[df.Minority == 1, 'Stop']
not_minority = df.loc[df.Minority == 0, 'Stop']

stats.ttest_ind(minority, not_minority)

### Simulation 2: Assign higher probability of stop to minority group 

In [ ]:
NUM_DRIVERS = 100000
PCT_MINORITY = 0.1
VAR_NOTMINOR = 1.0
VAR_MINORITY = 1.2
STOP_THRESH =  -2

n_minority = int(NUM_DRIVERS * PCT_MINORITY)
n_notminor = int(NUM_DRIVERS * (1 - PCT_MINORITY))

# Generate data
M = np.array([0] * n_notminor + [1] * n_minority) # generate minority variable

# generate error terms: for minority, variance is higher mean is the same.
N_MINORITY = np.random.normal(0, VAR_MINORITY, size=int(NUM_DRIVERS * PCT_MINORITY))
N_NOTMINOR = np.random.normal(0, VAR_NOTMINOR, size=int(NUM_DRIVERS * (1 - PCT_MINORITY)))
N = np.append(N_NOTMINOR, N_MINORITY)

df = pd.DataFrame({'Minority': M, 'StopLikelihood': N})

df['Stop'] = (df['StopLikelihood'] < STOP_THRESH).astype(int) # a stop occurs...
df.head()

In [ ]:
df.pivot_table(values='Stop', index='Minority', aggfunc='mean')

Testing the average difference in stop rates between the two groups:

In [ ]:
stats.ttest_ind(df[df.Minority==1]['Stop'],df[df.Minority==0]['Stop'])

## Bias

### Simulation 3: Adding the neighborhood of stop

* Probability of Stop: Neighborhood predicts likelihood of stop
* Racial distribution by neighborhood unequal
        

$$  Y_i=f(M_i, N_i, \varepsilon_i) $$
$$ \varepsilon \sim N(\mathbf{0},\mathbf{1}) $$

#### What happens if we test the same null hypothesis the same way?

In [ ]:
# Generate data per neighborhood

STOP_THRESH1 = -2
STOP_THRESH2 = -0.5

# Neighborhood 1: 10% minority; 100,000 people.

M1 = np.array([1] * 10000 + [0] * 90000) # generate minority variable
N1 = np.random.normal(0, 1, size=100000) # generate error terms
df1 = pd.DataFrame({'Minority': M1, 'StopLikelihood': N1, 'Neighborhood': 1})

df1['Stop'] = (df1['StopLikelihood'] < STOP_THRESH1).astype(int) # a stop occurs...

# Neighborhood 2: 50% minority; 100,000 people.

M2 = np.array([0] * 50000 + [1] * 50000) # generate minority variable
N2 = np.random.normal(0, 1, size=100000) # generate error terms
df2 = pd.DataFrame({'Minority': M2, 'StopLikelihood': N2, 'Neighborhood': 2})

df2['Stop'] = (df2['StopLikelihood'] < STOP_THRESH2).astype(int) # a stop occurs...

df = pd.concat([df1, df2])

In [ ]:
df.pivot_table(values='Stop',index=['Neighborhood','Minority'], aggfunc=np.mean)

T-test in neighborhood 1

In [ ]:
## Ttest Neighborhood 1

stats.ttest_ind(
    df.loc[(df.Minority==1) & (df.Neighborhood==1), 'Stop'],
    df.loc[(df.Minority==0) & (df.Neighborhood==1), 'Stop']
)

T-test in neighborhood 2

In [ ]:
## Ttest Neighborhood 2

stats.ttest_ind(
    df.loc[(df.Minority==1) & (df.Neighborhood==2), 'Stop'],
    df.loc[(df.Minority==0) & (df.Neighborhood==2), 'Stop']
)

#### Testing results  when combining neighborhoods

In [ ]:
pd.pivot_table(df, values='Stop',index=['Minority'], aggfunc=np.mean)

In [ ]:
stats.ttest_ind(
    df.loc[(df.Minority==1), 'Stop'],
    df.loc[(df.Minority==0), 'Stop']
)

### Probability of Stop: adding age of driver

* Likelihood of stop increases as driver age increases.
* Racial distribution by age of driver is unequal.

In [ ]:
# function to generate ages
def randomskew(skewness, maxVal, N):
    maxVal = maxVal - 16
    random = stats.skewnorm.rvs(a=skewness,loc=maxVal, size=N)
    random = random - min(random)      # Shift the set so the minimum value is equal to zero.
    random = random / max(random)      # Standadize all the values between 0 and 1. 
    random = random * maxVal + 16
    return random

In [ ]:
M1 = np.array([1] * 10000 + [0] * 90000) # generate minority variable
N1 = np.random.normal(0, 1, size=100000) # generate error terms
df1 = pd.DataFrame({'Minority': M1, 'StopLikelihood': N1, 'Neighborhood': 1})

df1['Stop'] = (df1['StopLikelihood'] < STOP_THRESH1).astype(int) # a stop occurs...


In [ ]:
M1 = np.array([1] * 10000 + [0] * 90000)
a = np.append(randomskew(0, 80, 10000), randomskew(4, 80, 90000))
lr = np.random.normal(-0.1, 1, 100000)
df = pd.DataFrame({'Minority': M1, 'StopLikelihood': lr, 'Age': a})
df['Stop'] =  ((df['StopLikelihood'] + df['Age'] * -0.01) < -2).astype('int')

In [ ]:
df.pivot_table(values='Stop',index=['Minority'], aggfunc=np.mean)

In [ ]:
minority = df.loc[df.Minority == 1, 'Stop']
non_minority = df.loc[df.Minority==0, 'Stop']
stats.ttest_ind(minority, non_minority)

In [ ]:
X = np.array(df[['Minority','Age']])
y = np.array(df.Stop)
X = sm.add_constant(X)
ols = sm.OLS(y, X)
ols_result = ols.fit()
ols_result.params

In [ ]:
print(ols_result.summary())

In [ ]:
df.groupby('Minority').Age.plot(kind='hist', legend=True);